In [0]:
# common code
dbutils.widgets.text("username", "", "MongoDB username")
dbutils.widgets.text("password", "", "MongoDB password")
dbutils.widgets.text("cluster", "", "MongoDB cluster")
username = dbutils.widgets.get("username")
password = dbutils.widgets.get("password")
cluster = dbutils.widgets.get("cluster")


In [0]:
#how to use mongodb client to operate the documentDB
%pip install pymongo

from pymongo import MongoClient
from pymongo.errors import ConnectionFailure
from urllib.parse import quote_plus

import sys


def main():
    client = None
    try:
        # Connection string for Azure DocumentDB cluster
        encoded_username = quote_plus(username)  # if the username contains special characters, encode it
        encoded_password = quote_plus(password)  # if the password contains special characters, encode it
        connection_string = f"mongodb+srv://{encoded_username}:{encoded_password}@{cluster}.global.mongocluster.cosmos.azure.com/?tls=true&authMechanism=SCRAM-SHA-256&retrywrites=false&maxIdleTimeMS=120000"

        print(f"Connection string: {connection_string}")
        # Create a new client and connect to the server
        client = MongoClient(connection_string)
        # Ping the server to verify connection
        client.admin.command("ping")
        print("Successfully connected and pinged Azure DocumentDB")
        # Get database reference
        database = client["adventureworks"]
        print(f"Connected to database: {database.name}")
        # Get collection reference
        collection = database["products"]
        print("Connected to collection: products")

        # Create sample products
        products = [
            {
                "_id": "00000000-0000-0000-0000-000000004018",
                "name": "Windry Mittens",
                "category": "apparel-accessories-gloves-and-mittens",
                "quantity": 121,
                "price": 35.00,
                "sale": False,
            },
            {
                "_id": "00000000-0000-0000-0000-000000004318",
                "name": "Niborio Tent",
                "category": "gear-camp-tents",
                "quantity": 140,
                "price": 420.00,
                "sale": True,
            },
        ]

        # Insert documents with upsert
        for product in products:
            filter_doc = {"_id": product["_id"]}
            collection.replace_one(filter_doc, product, upsert=True)
            print(f"Upserted product: {product['name']}")

        # Retrieve a specific document by ID
        filter_doc = {"_id": "00000000-0000-0000-0000-000000004018"}
        retrieved_product = collection.find_one(filter_doc)

        if retrieved_product:
            print(
                f"Retrieved product: {retrieved_product['name']} - ${retrieved_product['price']}"
            )
        else:
            print("Product not found")

        # Query for products on sale
        sale_filter = {"sale": True}
        sale_products = list(collection.find(sale_filter))
        print("Products on sale:")
        for product in sale_products:
            print(
                f"- {product['name']}: ${product['price']:.2f} (Category: {product['category']})"
            )

    except ConnectionFailure as e:
        print(f"Could not connect to MongoDB: {e}")
        sys.exit(1)
    except Exception as e:
        print(f"An error occurred: {e}")
        sys.exit(1)
    finally:
        if client is not None:
            client.close()


if __name__ == "__main__":
    main()